In [1]:
import pandas as pd

In [2]:
drugs_df = pd.read_csv('de.csv')

In [3]:
drugs_df[:3]

,DATE,STATUS,QLANG,DEM_GENDER,DEM_AGE,DEM_LOCATION,DEM_POSTAL,DEM_STDNT,DEM_VET,DEM_HEALTH,...,COD_NMU_NTY,DIHY_NMU_NTY,FENT_NMU_NTY,HYDM_NMU_NTY,METH_NMU_NTY,MORPH_NMU_NTY,OXY_NMU_NTY,SUF_NMU_NTY,TAP_NMU_NTY,TRAM_NMU_NTY
0,2018-01-12 08:08:01,3,2,1,21,1,786,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2018-01-14 08:08:30,3,2,1,24,1,797,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2018-01-14 07:31:08,3,2,1,19,1,717,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
def misuse_cat(df):
    # make new df so old dataset is not affected
    new_df = drugs_df.copy()
    # get list of drug names
    drugs = [x for x in df.columns if 'NMU' in x][:14]
    drugs = [x.split('_')[0] for x in drugs]
    print(drugs)
    # make categorical variable for each drug
    for drug in drugs:
        new_df[f'{drug}_MISUSE'] = new_df[f'{drug}_NMU'].fillna(0)
        new_df[f'{drug}_MISUSE'] = new_df[f'{drug}_MISUSE'].astype(int)
    # in the returned df, each drug now has a column indicating how the correspondent uses the drug
    # 0   -> never used
    # 1   -> used for perscription purposes
    # 2   -> used for recreational purposes
    return new_df, drugs

In [6]:
drugs_df, drugs = misuse_cat(drugs_df)

['FENT', 'BUP', 'METH', 'MORPH', 'OXY', 'TRAM', 'TAP', 'SUF', 'COD', 'DIHY', 'HYDM', 'STIM', 'BENZ', 'THC']


In [7]:
def misuse_total(df):
    new_df = drugs_df.copy()
    new_df['MIS_TOTAL'] = 0
    for drug in drugs:
        new_df['MIS_TOTAL'] = new_df['MIS_TOTAL'] + new_df[f'{drug}_MISUSE']
    return new_df

In [10]:
drugs_df = misuse_total(drugs_df)

In [11]:
Misuse_var = []
for drug in drugs:
    Misuse_var.append(f'{drug}_MISUSE')

In [12]:
def get_use_cat(df):
    # make new df so old dataset is not affected
    new_df = drugs_df.copy()
    # get list of drug names
    drugs = [x for x in df.columns if 'NMU' in x][:14]
    drugs = [x.split('_')[0] for x in drugs]
    # make categorical variable for each drug
    for drug in drugs:
        new_df[f'{drug}_USE_CAT'] = new_df[f'{drug}_USE'] + new_df[f'{drug}_NMU'].fillna(0)
        new_df[f'{drug}_USE_CAT'] = new_df[f'{drug}_USE_CAT'].astype(int)
    # in the returned df, each drug now has a column indicating how the correspondent uses the drug
    # 0   -> never used
    # 1   -> used for perscription purposes
    # 2   -> used for recreational purposes
    return new_df

In [ ]:
drugs_df = get_use_cat(drugs_df)

In [14]:
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

In [15]:
Misuse_var

['FENT_MISUSE',
 'BUP_MISUSE',
 'METH_MISUSE',
 'MORPH_MISUSE',
 'OXY_MISUSE',
 'TRAM_MISUSE',
 'TAP_MISUSE',
 'SUF_MISUSE',
 'COD_MISUSE',
 'DIHY_MISUSE',
 'HYDM_MISUSE',
 'STIM_MISUSE',
 'BENZ_MISUSE',
 'THC_MISUSE']

In [16]:
y_var_list = ['MIS_TOTAL', 'DRSHOP_USE']
X_variables = ['DEM_GENDER', 'DEM_AGE10', 'DEM_STDNT', 'DEM_MARITAL', 'DEM_EDU', 'TOB_USE', 'ALC_USE', 'MENT_BIP']

In [18]:
clf = tree.DecisionTreeClassifier()
for y_var in y_var_list:
    y = drugs_df[y_var]
    X = drugs_df[X_variables]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=42)
    
    clf = clf.fit(X_train, y_train)
    
    # Accuracy of the Training set
    X_train_predict = clf.predict(X_train)
    print("Accuracy of the training set is", sum(y_train == X_train_predict)/len(X_train))
    
    # Accuracy of the Testing set
    X_test_predict = clf.predict(X_test)
    print("Accuracy of the testing set is", sum(y_test == X_test_predict)/len(X_test))

Accuracy of the training set is 0.8344407530454042
Accuracy of the testing set is 0.7618335824613851
Accuracy of the training set is 0.9891472868217054
Accuracy of the testing set is 0.981398438797542


In [ ]:
# Accuracy of the Training set
X_train_predict = clf.predict(X_train)
sum(y_train == X_train_predict)/len(X_train)

In [ ]:
# Accuracy of the Testing set
X_test_predict = clf.predict(X_test)
sum(y_test == X_test_predict)/len(X_test)

In [ ]:
# Neural Network
clf_nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 3), random_state=1)
clf_nn.fit(X_train, y_train)

In [ ]:
# Accuracy of the Training set
X_train_predict_nn = clf_nn.predict(X_train)
sum(y_train == X_train_predict_nn)/len(X_train)

In [ ]:
# Accuracy of the Testing set
X_test_predict_nn = clf_nn.predict(X_test)
sum(y_test == X_test_predict_nn)/len(X_test)